<a href="https://colab.research.google.com/github/TomAmster/Data-Science/blob/master/Data_Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np

In [0]:
drive_dir = '/content/drive/Shared drives/DS Workshop/data/'
train_path = drive_dir + 'train.csv'
train = pd.read_csv(train_path)
bids_path = drive_dir + 'bids.csv/bids.csv'
bids = pd.read_csv(bids_path)
test_path = drive_dir + 'test.csv'
test = pd.read_csv(test_path)

In [0]:
bid = pd.merge(train, bids, on='bidder_id', how='left')

# **Data Preprocessing**

## Data Cleaning

### Checking Missing Values

In [0]:
pd.isnull(bid).sum()

bidder_id             0
payment_account       0
address               0
outcome               0
bid_id               29
auction              29
merchandise          29
device               29
time                 29
country            2730
ip                   29
url                  29
dtype: int64

### Check for bidder without bids

In [0]:
nobids_train = bid[pd.isnull(bid['bid_id'])]['bidder_id'].unique()
print("amout of bidders without bids",len(nobids_train))


amout of bidders without bids 29


### Drop Bidders without bids

In [0]:
bid = bid[~bid['bidder_id'].isin(nobids_train)]

In [0]:
pd.isnull(bid).sum()

bidder_id             0
payment_account       0
address               0
outcome               0
bid_id                0
auction               0
merchandise           0
device                0
time                  0
country            2701
ip                    0
url                   0
dtype: int64

In [0]:
count_miss_ratio = pd.isnull(bid['country']).sum()/len(bid)*100
print(f" %0.2f %% observations missing country entry." %
      float(count_miss_ratio))
print()

 0.09 % observations missing country entry.


In [0]:
bid = bid.dropna()
pd.isnull(bid).any()


bidder_id          False
payment_account    False
address            False
outcome            False
bid_id             False
auction            False
merchandise        False
device             False
time               False
country            False
ip                 False
url                False
dtype: bool

## Outliers

###TO BE FILLED
https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

## Data Embedding & Representation

### Representing Address and Payment Accounts with Bins

In [0]:
### Strip last 5 digits from payment__account and address
temp_train = train
temp_train.address = temp_train.address.apply(lambda x: x[:-5])
temp_train.payment_account = temp_train.payment_account.apply(lambda x: x[:-5])

## DO THE SAME FOR TEST
temp_test = test
temp_test.address = temp_test.address.apply(lambda x: x[:-5])
temp_test.payment_account = temp_test.payment_account.apply(lambda x: x[:-5])

## Create Feature - is payment account and address are equal
temp_train['payment_account_prefix_same_as_address_prefix'] = temp_train['address']==temp_train['payment_account']
temp_test['payment_account_prefix_same_as_address_prefix'] = temp_test['address']==temp_test['payment_account']

## Create Feature from Address - 
## Convert Rare and infrequent addresses to bins 
## Create One-Hot Encoding of the Rest of the Addresses
temp = temp_train[['bidder_id', 'address']].groupby('address').size().reset_index().sort_values(0, ascending=False)
temp = 	temp.rename(columns = {0:'address_count'})
infrequent_addresses = temp[temp.address_count >= 2][temp.address_count < 6].address.values
rare_addresses = temp[temp.address_count < 2].address.values
temp_train['address'].loc[temp_train.address.isin(rare_addresses)] = 'rare_address'
temp_train['address'].loc[temp_train.address.isin(infrequent_addresses)] = 'infrequent_address'
temp = pd.get_dummies(temp_train[['address']])
temp_train = pd.concat([temp_train, temp], axis=1)
###-------------
temp2 = temp_test[['bidder_id', 'address']].groupby('address').size().reset_index().sort_values(0, ascending=False)
temp2 = temp2.rename(columns = {0:'address_count'})
infrequent_addresses2 = temp2[temp2.address_count >= 2][temp2.address_count < 6].address.values
rare_addresses2= temp2[temp2.address_count < 2].address.values
temp_test['address'].loc[temp_test.address.isin(rare_addresses2)] = 'rare_address'
temp_test['address'].loc[temp_test.address.isin(infrequent_addresses2)] = 'infrequent_address'
temp2 = pd.get_dummies(temp_test[['address']])
temp_test = pd.concat([temp_test, temp2], axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [0]:
## Create Feature from Payment Accounts - 
## Convert Rare and infrequent Payment Accounts to bins 
## Create One-Hot Encoding of the Rest of the Payment Accounts
temp = temp_train[['bidder_id', 'payment_account']].groupby('payment_account').size().reset_index().sort_values(0, ascending=False)
temp = temp.rename(columns = {0:'payment_account_count'})
temp2 = temp_test[['bidder_id', 'payment_account']].groupby('payment_account').size().reset_index().sort_values(0, ascending=False)
temp2 = temp2.rename(columns = {0:'payment_account_count'})

infrequent_payment_account = temp[temp.payment_account_count >= 2][temp.payment_account_count < 6].payment_account.values
rare_payment_account = temp[temp.payment_account_count < 2].payment_account.values
temp_train['payment_account'].loc[temp_train.payment_account.isin(rare_payment_account)] = 'rare_account'
temp_train['payment_account'].loc[temp_train.payment_account.isin(infrequent_payment_account)] = 'infrequent_account'	
temp =	pd.get_dummies(temp_train[['payment_account']])
temp_train = pd.concat([temp_train, temp], axis=1)

infrequent_payment_account2 = temp2[temp2.payment_account_count >= 2][temp2.payment_account_count < 6].payment_account.values
rare_payment_account2 = temp2[temp2.payment_account_count < 2].payment_account.values
temp_test['payment_account'].loc[temp_test.payment_account.isin(rare_payment_account2)] = 'rare_account'
temp_test['payment_account'].loc[temp_test.payment_account.isin(infrequent_payment_account2)] = 'infrequent_account'	
temp2 =	pd.get_dummies(temp_test[['payment_account']])
temp_test = pd.concat([temp_test, temp2], axis=1)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


## One-Hot Encoding Merchandise Features - Categorical Information

In [0]:
data_train =pd.merge(temp_train,bids,on='bidder_id', how='left') 
data_test = pd.merge(temp_test,bids,on='bidder_id', how='left')

In [0]:
print(data_train.merchandise.value_counts())
print(data_test.merchandise.value_counts())

sporting goods      1169724
mobile               597488
jewelry              592735
home goods           407957
office equipment     168638
furniture             87807
computers             21400
books and music       15242
auto parts             9757
clothing                476
Name: merchandise, dtype: int64
mobile              1529099
jewelry             1309323
home goods           816277
sporting goods       685483
office equipment     121200
computers             59684
books and music       36699
clothing              15971
furniture             11374
Name: merchandise, dtype: int64


In [0]:
a = data_train.groupby('bidder_id').first().reset_index()
b = pd.get_dummies(a['merchandise'], columns='merchandise')
data_train = pd.concat([bid, b], axis=1)


a = data_test.groupby('bidder_id').first().reset_index()
b = pd.get_dummies(a['merchandise'], columns='merchandise')
data_test= pd.concat([data_test, b], axis=1)
